In [19]:

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from scipy import stats
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

review = pd.read_csv('/Users/aayushbhargava/Downloads/Dataset/train.csv')
review_test = pd.read_csv('/Users/aayushbhargava/Downloads/Dataset/test.csv')
review.head()

,Review Text,Review Title,topic
0,"Did nothing for me, didn't help lost even with...",Useless,Shipment and delivery
1,"Did nothing for me, didn't help lost even with...",Useless,Not Effective
2,I have bought these bags and immediately open...,TRASH!!! Do not buy these bags it’s a waist of...,Customer Service
3,Gave me an allergic reaction on my face :(,Do not recommend,Allergic
4,These don't compare to the name brand wipes. F...,Can't tackle big messes,Texture


In [21]:
# Check if any row is null
print (review.columns[review.isnull().any()])
print (review_test.columns[review_test.isnull().any()])

Index([], dtype='object')
Index([], dtype='object')


In [3]:
review['topic'].unique()

array(['Shipment and delivery', 'Not Effective', 'Customer Service',
       'Allergic', 'Texture', 'Quality/Contaminated', 'Color and texture',
       'Bad Taste/Flavor', 'Too big to swallow', 'Smells Bad',
       'Too Sweet', 'Ingredients', 'Expiry', 'Packaging',
       'Wrong Product received', 'Pricing', 'False Advertisement',
       'Inferior to competitors', "Didn't Like", 'Customer Issues',
       'Hard to Chew'], dtype=object)

In [4]:
#Cleaning data (removing stopwords, punctuations or whatever, converting to lower case)
def fc(tab_name):
    stemmer = SnowballStemmer("english")
    def cleanerr(tbc):
        stop_words = set(stopwords.words('english'))
        for num,x in enumerate(tab_name[tbc]):
            x=re.sub(r'[^\w\s]','',x)
            word_tokens = word_tokenize(x)
            filtered_sentence = [stemmer.stem(w.lower()) for w in word_tokens if not w in stop_words and len(w)>=3]
            tab_name[tbc].iloc[num]=filtered_sentence
            tab_name[tbc].iloc[num]=" ".join(tab_name[tbc].iloc[num])
    cleanerr('Review Text')
    cleanerr('Review Title')
    tab_name.head()

    tab_name['Review Text']=tab_name['Review Text']+' '+tab_name['Review Title']
    tab_name['Review Text'].iloc[0]

fc(review)
fc(review_test)

In [5]:
gg=review['Review Text']
print(type(gg))
#gg.append(review_test['Review Text'], ignore_index=True)
gg=pd.concat([gg,review_test['Review Text']], ignore_index=True)
print (review['Review Text'].shape)
gg.tail()

<class 'pandas.core.series.Series'>
(5959,)


8507    this wors protein ive ever tri the tast horrib...
8508    veri small easi swallow flavor left stomach kn...
8509    veri small easi swallow flavor left stomach kn...
8510    good increas bad cholesterol ldlbr doesnt suit...
8511                will buy powder thicklik past consist
Name: Review Text, dtype: object

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# create a count vectorizer object
count_vectorizer = CountVectorizer()
# fit the count vectorizer using the text data
count_vectorizer.fit(gg)
# collect the vocabulary items used in the vectorizer
dictionary = count_vectorizer.vocabulary_.items() 
len(dictionary)

8737

In [8]:
import numpy as np

tfid_vectorizer = TfidfVectorizer("english")
tfid_vectorizer.fit(gg)
tfid_matrix = tfid_vectorizer.transform(review['Review Text'])
array_text = tfid_matrix.todense()

tfid_matrix = tfid_vectorizer.transform(review_test['Review Text'])
array_text_test = tfid_matrix.todense()

#To inverse_transform label encoders
le = LabelEncoder()
rto = le.fit_transform(review['topic'])
labels=np.array(rto)


In [ ]:
#To be used to save and retrive array for faster access
'''
np.save('/root/Documents/rev_text.npy', array_text)
np.save('/root/Documents/rev_title.npy', array_title)
np.save('/root/Documents/labels.npy', labels)

array_text = np.load('/root/Documents/rev_text.npy')
array_title = np.load('/root/Documents/rev_title.npy')
labels = np.load('/root/Documents/labels.npy')
'''

In [9]:
r=int(array_text.shape[0]*0.90)
traffic_lab=array_text[:r]
label_lab=labels[:r]

traffic_val=array_text[r:]
label_val=labels[r:]

In [10]:
traffic_lab.shape

(5363, 8737)

In [11]:
from xgboost import XGBClassifier

xgb=XGBClassifier(objective='multi:softmax',learning_rate=0.01,n_class=5, max_depth=2)
print (xgb)
xgb.fit(traffic_lab,label_lab,eval_set = [(traffic_val, label_val)],verbose=True,early_stopping_rounds=5)
tt=xgb.predict(traffic_val)
accuracy_lgbm = accuracy_score(tt,label_val)
print (accuracy_lgbm)

#filename = '/root/Documents/xgboot_model.sav'
#joblib.dump(lgbm, filename)

for i in range(1,20):
    tt=(xgb.predict(traffic_val[i]))
    print (str(label_val[i])+'\t'+str(tt))

for i in range(1,20):
    tt=np.argsort(-xgb.predict_proba(traffic_val[i]))
    print (str(label_val[i])+'\t'+str(tt))


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_class=5, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
[0]	validation_0-merror:0.446309
Will train until validation_0-merror hasn't improved in 5 rounds.
[1]	validation_0-merror:0.447987
[2]	validation_0-merror:0.447987
[3]	validation_0-merror:0.447987
[4]	validation_0-merror:0.444631
[5]	validation_0-merror:0.442953
[6]	validation_0-merror:0.441275
[7]	validation_0-merror:0.442953
[8]	validation_0-merror:0.441275
[9]	validation_0-merror:0.439597
[10]	validation_0-merror:0.437919
[11]	validation_0-merror:0.436242
[12]	validation_0-merror:0.436242
[13]	validation_0-merr

NameError: name 'accuracy_score' is not defined

In [12]:
for i in range(0,60):
    tt=(xgb.predict(traffic_val[i]))
    print (str(label_val[i])+'\t'+str(tt))

for i in range(0,60):
    tt=np.argsort(-xgb.predict_proba(traffic_val[i]))
    print (str(label_val[i])+'\t'+str(tt))


0	[0]
9	[14]
12	[12]
19	[0]
14	[14]
0	[14]
15	[15]
11	[10]
10	[10]
17	[17]
1	[17]
18	[18]
10	[18]
1	[1]
1	[1]
4	[1]
5	[1]
14	[1]
1	[1]
11	[1]
1	[1]
4	[1]
10	[0]
0	[0]
0	[0]
14	[14]
11	[14]
14	[1]
1	[1]
1	[1]
17	[17]
17	[14]
14	[14]
13	[0]
15	[15]
11	[11]
19	[0]
14	[1]
1	[1]
9	[9]
17	[17]
1	[1]
0	[0]
11	[11]
19	[12]
17	[12]
14	[12]
11	[12]
12	[12]
17	[6]
6	[6]
4	[4]
11	[1]
10	[10]
17	[17]
17	[1]
0	[1]
1	[1]
17	[19]
19	[19]
0	[[ 0 11  1 12 15 10 17 19 14 13 16  2  4 18  5 20  9  6  7  8  3]]
9	[[14  0 11  1 12 15 10 17 19 13 16  2  4 18  5 20  9  6  7  8  3]]
12	[[12  4  0 11  1 15 10 17 19 14 13 16  2 18  5 20  9  6  7  8  3]]
19	[[ 0 11  1 12 15 10 17 19 14 13 16  2  4 18  5 20  9  6  7  8  3]]
14	[[14  0 11  1 12 15 10 17 19 13 16  2  4 18  5 20  9  6  7  8  3]]
0	[[14  0 11  1 12 15 10 17 19 13 16  2  4 18  5 20  9  6  7  8  3]]
15	[[15 20  4  0 11  1 12 10 17 19 14 13 16  2 18  5  9  6  7  8  3]]
11	[[10  0 11  1 12 15 17 19 14 13 16  2  4 18  5 20  9  6  7  8  3]]
10	[[10  0 11  1 

In [20]:
count=0
for i in range(0,59):
        tt=(xgb.predict(array_text_test[i]))
        print (str(le.inverse_transform(tt)))

['Bad Taste/Flavor']
['Bad Taste/Flavor']
['Allergic']
['Ingredients']
['Ingredients']
['Quality/Contaminated']
['Quality/Contaminated']
['Quality/Contaminated']
['Quality/Contaminated']
['Shipment and delivery']
['Not Effective']
['Packaging']
['Allergic']
['Allergic']
['Bad Taste/Flavor']
['Bad Taste/Flavor']
['Not Effective']
['Not Effective']
['Bad Taste/Flavor']
['Allergic']
['Color and texture']
['Color and texture']
['Customer Service']
['Quality/Contaminated']
['Quality/Contaminated']
['Quality/Contaminated']
['Packaging']
['Not Effective']
['Not Effective']
['Quality/Contaminated']
['Quality/Contaminated']
['Packaging']
['Packaging']
['Packaging']
['Packaging']
['Packaging']
['Expiry']
['Allergic']
['Allergic']
['Allergic']
['Bad Taste/Flavor']
['Bad Taste/Flavor']
['Shipment and delivery']
['Shipment and delivery']
['Allergic']
['Bad Taste/Flavor']
['Bad Taste/Flavor']
['Bad Taste/Flavor']
['Wrong Product received']
['Wrong Product received']
['Packaging']
['Allergic']
['Alle

In [18]:
#for i in range(0,10):
#    tt=np.argsort(-xgb.predict_proba(array_text_test[i]))
#    print (str(tt))
import csv

f = open("/Users/aayushbhargava/Downloads/Dataset/Submission.csv", "a")
fff=csv.writer(f)
fff.writerow(['Review Text','Review Title','topic'])
count=0
for i in range(0,len(array_text_test)):
    try:
        if (array_text_test[i]==array_text_test[i+1]).all():
            tt=np.argsort(-xgb.predict_proba(array_text_test[i]))
            tt=(str(le.inverse_transform([tt[0][count]])))
            fff.writerow([review_test['Review Text'].iloc[i],review_test['Review Title'].iloc[i], str(tt).replace("['",'').replace("']",'')])
            count=count+1
        else:
            if count==0:
                tt=(xgb.predict(array_text_test[i]))
                fff.writerow([review_test['Review Text'].iloc[i],review_test['Review Title'].iloc[i],str(le.inverse_transform(tt)).replace("['",'').replace("']",'')])
            else:
                tt=np.argsort(-xgb.predict_proba(array_text_test[i]))
                tt=(str(le.inverse_transform([tt[0][count]])))
                fff.writerow([review_test['Review Text'].iloc[i],review_test['Review Title'].iloc[i],str(tt).replace("['",'').replace("']",'')])
                count=0
    except:
        tt=(xgb.predict(array_text_test[i]))
        fff.writerow([review_test['Review Text'].iloc[i],review_test['Review Title'].iloc[i],str(le.inverse_transform(tt)).replace("['",'').replace("']",'')])
f.close()
print ('==================All Done')

==================All Done
